In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [2]:
def plot_horizontal_projection(file_name, img, projection):
    fig = plt.figure(1, figsize=(12,16))
    gs = gridspec.GridSpec(1, 2, width_ratios=[3,1])

    ax = plt.subplot(gs[0])
    im = ax.imshow(img, interpolation='nearest', aspect='auto')
    ax.grid(which='major', alpha=0.5)

    ax = plt.subplot(gs[1])
    ax.plot(projection, np.arange(img.shape[0]), 'm')
    ax.grid(which='major', alpha=0.5)
    plt.xlim([0.0, 255.0])
    plt.ylim([-0.5, img.shape[0] - 0.5])
    ax.invert_yaxis()

    fig.suptitle("FOO", fontsize=16)
    gs.tight_layout(fig, rect=[0, 0.03, 1, 0.97])  

    fig.set_dpi(200)

    fig.savefig(file_name, bbox_inches='tight', dpi=fig.dpi)
    plt.clf() 

def plot_vertical_projection(file_name, img, projection):
    fig = plt.figure(2, figsize=(12, 4))
    gs = gridspec.GridSpec(2, 1, height_ratios=[1,5])

    ax = plt.subplot(gs[0])
    im = ax.imshow(img, interpolation='nearest', aspect='auto')
    ax.grid(which='major', alpha=0.5)

    ax = plt.subplot(gs[1])
    ax.plot(np.arange(img.shape[1]), projection, 'm')
    ax.grid(which='major', alpha=0.5)
    plt.xlim([-0.5, img.shape[1] - 0.5])
    plt.ylim([0.0, 255.0])

    fig.suptitle("FOO", fontsize=16)
    gs.tight_layout(fig, rect=[0, 0.03, 1, 0.97])  

    fig.set_dpi(200)

    fig.savefig(file_name, bbox_inches='tight', dpi=fig.dpi)
    plt.clf() 

def visualize_hp(file_name, img, row_means, row_cutpoints):
    row_highlight = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    row_highlight[row_means == 0, :, :] = [255,191,191]
    row_highlight[row_cutpoints.astype('int64'), :, :] = [255,0,0]
    plot_horizontal_projection(file_name, row_highlight, row_means)

def visualize_vp(file_name, img, column_means, column_cutpoints):
    col_highlight = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    col_highlight[:, column_means == 0, :] = [255,191,191]
    col_highlight[:, column_cutpoints.astype('int64'), :] = [255,0,0]
    plot_vertical_projection(file_name, col_highlight, column_means)


# From https://stackoverflow.com/a/24892274/3962537
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges



In [16]:
img = cv2.imread('UOB-Sustainability-Report-2020_Output/page36.png', cv2.IMREAD_COLOR) #extract img
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # turn img to grey
img_gray_inverted = 255 - img_gray #

row_means = cv2.reduce(img_gray_inverted, 1, cv2.REDUCE_AVG, dtype=cv2.CV_32F).flatten()
row_gaps = zero_runs(row_means)
row_cutpoints = (row_gaps[:,0] + row_gaps[:,1] - 1) / 2

visualize_hp("article_img11.png", img, row_means, row_cutpoints)

<Figure size 2400x3200 with 0 Axes>

In [10]:
#cv2.imshow('image',img_gray)
#cv2.waitKey(0)

In [11]:

bounding_boxes = []
for n,(start,end) in enumerate(zip(row_cutpoints, row_cutpoints[1:])):
    line = img[int(start):int(end)]
    line_gray_inverted = img_gray_inverted[int(start):int(end)]

    column_means = cv2.reduce(line_gray_inverted, 0, cv2.REDUCE_AVG, dtype=cv2.CV_32F).flatten()
    column_gaps = zero_runs(column_means)
    column_gap_sizes = column_gaps[:,1] - column_gaps[:,0]
    column_cutpoints = (column_gaps[:,0] + column_gaps[:,1] - 1) / 2

    filtered_cutpoints = column_cutpoints[column_gap_sizes > 50] # this part can use ML too 

    for xstart,xend in zip(filtered_cutpoints, filtered_cutpoints[1:]):

        bounding_boxes.append(((int(xstart), int(start)), (int(xend), int(end))))

    # visualize_vp("PingAn images/Page 11/article_vp/11article_vp_%02d.png" % n, line, column_means, filtered_cutpoints)

result = img.copy()
# print(bounding_boxes)

In [12]:
# This part can use ML to determine where to slice 

def checkDim(height, width, area):
    print(f"{height} x {width} = {area}")
    if(height <= 200):
        return False
    if(width <= 200):
        return False
    if(area <= 100000):
        return False
    return True
    

In [15]:
count = 0
for bounding_box in bounding_boxes:
    count = count + 1
    
    print("==================== " + str(count))
    print(bounding_box[0])
    print(bounding_box[1])
    print(f"x:end = {bounding_box[0][0]}")
    print(f"x:start = {bounding_box[0][1]}")
    print(f"y:end = {bounding_box[1][0]}")
    print(f"y:start = {bounding_box[1][1]}")    

    
    height = bounding_box[1][1]-bounding_box[0][1]
    width = bounding_box[1][0]-bounding_box[0][0]
    area = height * width
    print(f"{height} x {width} = {area}")
    
    if(checkDim(height, width, area) == True):
        print("FOUNDED")
        x = 2.2
        scale_horizontal = x*64
        scale_vertical = x*64
        new_x = (int(bounding_box[0][0])-int(scale_horizontal),int(bounding_box[0][1])-int(scale_vertical))
        new_y = (int(bounding_box[1][0])+int(scale_horizontal),int(bounding_box[1][1])+int(scale_vertical))

        cv2.rectangle(result, new_x, new_y, (0, 0, 255), 2)
        cv2.imwrite("PingAn images/10/part%d.png" % count, result)
   
    

In [8]:
cv2.imwrite("Page10_output.png", result)

True